# Inteligencia artificial 2024-1
## UNIDAD 3 - Introducción a Deep Learning

#### Código fuente y contenido basado en:
`MIT: Introduction to Deep Learning`

Ejecutar en Google Colab	Ver código fuente en GitHub



<table align="center">

<td align="center"><a target="_blank" href="https://github.com/ulises1229/IA-2021-II/blob/main/code/IA_Unidad_4_Computer_Vision_y_DL.ipynb">
        <img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;" />Ejecutar en Google Colab</a></td>
  <td align="center"><a target="_blank" href="https://colab.research.google.com/drive/1XqqtBBWcEHGFIPD4_6lDSEnfNRJVTmcI?usp=sharing">
        <img src="https://i.ibb.co/xfJbPmL/github.png"  height="70px" style="padding-bottom:5px;"  />Ver código fuente en GitHub</a></td>


</table>



#1. Clasificación de dígitos usando el set de datos MNIST

En la primer parte crearemos y entrenaremos una red neuronal convolucional (CNN) para la clasificación de dígitos escritos a mano del famoso conjunto de datos [MNIST](http://yann.lecun.com/exdb/mnist/). El conjunto de datos MNIST consta de 60,000 imágenes de entrenamiento y 10,000 imágenes de prueba. Nuestras clases son los dígitos 0-9.


In [ ]:
# Import Tensorflow 2.0
%tensorflow_version 2.x
import tensorflow as tf

!pip install mitdeeplearning
import mitdeeplearning as mdl

import matplotlib.pyplot as plt
import numpy as np
import random
from tqdm import tqdm

# Check that we are using a GPU, if not switch runtimes
#   using Runtime > Change Runtime Type > GPU
assert len(tf.config.list_physical_devices('GPU')) > 0

## 1.1 Conjunto de datos MNIST

Descarguemos y carguemos el conjunto de datos y mostremos algunas muestras aleatorias de él:

In [ ]:
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = (np.expand_dims(train_images, axis=-1)/255.).astype(np.float32)
train_labels = (train_labels).astype(np.int64)
test_images = (np.expand_dims(test_images, axis=-1)/255.).astype(np.float32)
test_labels = (test_labels).astype(np.int64)

Nuestro conjunto de formación se compone de imágenes en escala de grises de 28x28 de dígitos escritos a mano.

Visualicemos cómo se ven algunas de estas imágenes y sus correspondientes etiquetas de entrenamiento.

In [ ]:
plt.figure(figsize=(10,10))
random_inds = np.random.choice(60000,36)
for i in range(16):
    plt.subplot(4,4,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    image_ind = random_inds[i]
    plt.imshow(np.squeeze(train_images[image_ind]), cmap=plt.cm.binary)
    plt.xlabel(train_labels[image_ind])

## 1.2 Red neuronal para clasificación de dígitos escritos a mano

Primero construiremos una red neuronal simple que consta de dos capas completamente conectadas y aplicaremos esto a la tarea de clasificación de dígitos. Nuestra red finalmente generará una distribución de probabilidad sobre las clases de 10 dígitos (0-9). Esta primera arquitectura que estaremos construyendo se muestra a continuación:

![alt_text](https://raw.githubusercontent.com/aamini/introtodeeplearning/master/lab2/img/mnist_2layers_arch.png "CNN Architecture for MNIST Classification")

### Arquitectura de red neuronal completamente conectada
Para definir la arquitectura de esta primera red neuronal completamente conectada, una vez más usaremos la API de Keras y definiremos el modelo usando una red [`Sequential`](https://www.tensorflow.org/api_docs/python/tf/keras / modelos / secuencial). Observe cómo usamos primero una capa [`Flatten`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten), que aplana la entrada para que pueda introducirse en el modelo.


In [ ]:
def build_fc_model():
  fc_model = tf.keras.Sequential([
      # First define a Flatten layer
      tf.keras.layers.Flatten(),

      # '''TODO: Define the activation function for the first fully connected (Dense) layer.'''
      tf.keras.layers.Dense(128, activation=tf.nn.relu),
      # tf.keras.layers.Dense(128, activation= '''TODO'''),
        tf.keras.layers.Dense(128, activation=tf.nn.relu),
      # '''TODO: Define the second Dense layer to output the classification probabilities'''
      tf.keras.layers.Dense(10, activation=tf.nn.softmax)
      # [TODO Dense layer to output classification probabilities]

  ])
  return fc_model

model = build_fc_model()



A medida que avanzamos en la siguiente parte, es posible que se deseen realizar cambios en la arquitectura definida anteriormente.

Demos un paso atrás y pensemos en la red que acabamos de crear. La primera capa de esta red, `tf.keras.layers.Flatten`, transforma el formato de las imágenes de una matriz 2d (28 x 28 píxeles) a una matriz 1d de 28 * 28 = 784 píxeles. Puede pensar en esta capa como desapilar filas de píxeles en la imagen y alinearlas. No hay parámetros aprendidos en esta capa; solo reformatea los datos.

Una vez que los píxeles se aplanan, la red consta de una secuencia de dos capas `tf.keras.layers.Dense`. Estas son capas neuronales completamente conectadas. La primera capa "Densa" tiene 128 nodos (o neuronas). La segunda (y última) capa debe devolver una matriz de puntuaciones de probabilidad que sumen 1. Cada nodo contiene una puntuación que indica la probabilidad de que la imagen actual pertenezca a una de las clases de dígitos escritos a mano.

¡Eso define nuestro modelo totalmente conectado!

### Compilando el modelo

Antes de entrenar el modelo, necesitamos definir algunas configuraciones más. Estas se agregan durante el paso del modelo [`compile`](https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#compile):

* Función de pérdida *: define cómo medimos la precisión del modelo durante el entrenamiento. Como se cubrió en la conferencia, durante el entrenamiento queremos minimizar esta función, que "dirigirá" el modelo en la dirección correcta.
> * Optimizador *: define cómo se actualiza el modelo en función de los datos que ve y su función de pérdida.
> * Métricas *: aquí podemos definir las métricas utilizadas para monitorear los pasos de entrenamiento y prueba. En este ejemplo, veremos la * precisión *, la fracción de las imágenes que están clasificadas correctamente.

Comenzaremos usando un optimizador de descenso de gradiente estocástico (SGD) inicializado con una tasa de aprendizaje de 0.1. Dado que estamos realizando una tarea de clasificación categórica, queremos usar la [pérdida de entropía cruzada](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/sparse_categorical_crossentropy).

Se recomienda experimentar tanto con la elección del optimizador como con la tasa de aprendizaje y evaluar cómo estos afectan la precisión del modelo.

In [ ]:
'''TODO: Experiment with different optimizers and learning rates. How do these affect
    the accuracy of the trained model? Which optimizers and/or learning rates yield
    the best performance?'''
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1e-1),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


### Entrenando el modelo

Ahora estamos listos para entrenar nuestro modelo, lo que implicará introducir los datos de entrenamiento (`train_images` y ` train_labels`) en el modelo, y luego pedirle que aprenda las asociaciones entre imágenes y etiquetas. También necesitaremos definir el tamaño de batch y el número de épocas, o iteraciones sobre el conjunto de datos MNIST, durante el entrenamiento.

En la unidad pasada, vimos cómo podemos usar `GradientTape` para optimizar pérdidas y entrenar modelos con el método de descenso de gradiente estocástico. Después de definir la configuración del modelo en el paso `compile`, también podemos realizar el entrenamiento llamando al [`fit`](https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#fit) método que se instancia de la clase `Model`. Usaremos esto para entrenar nuestro modelo completamente conectado

In [ ]:
# Define the batch size and the number of epochs to use during training
BATCH_SIZE = 128
EPOCHS = 5

model.fit(train_images, train_labels, batch_size=BATCH_SIZE, epochs=EPOCHS, use_multiprocessing=True)

model.summary()

A medida que el modelo se entrena, se muestran las métricas de pérdida y precisión. Con cinco épocas y una tasa de aprendizaje de 0.01, este modelo completamente conectado debería lograr una precisión de aproximadamente 0.97 (o 97%) en los datos de entrenamiento.

### Evaluar la precisión en el conjunto de datos de prueba

Ahora que hemos entrenado el modelo, podemos pedirle que haga predicciones sobre un set de prueba que no haya visto antes. En este ejemplo, el tensor `test_images` comprende nuestro conjunto de datos de prueba. Para evaluar la precisión, podemos verificar si las predicciones del modelo coinciden con las etiquetas de la matriz `test_labels`.

Utilice el método [`evaluate`](https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#evaluate) para evaluar el modelo en el conjunto de datos de prueba.

In [ ]:
'''TODO: Use the evaluate method to test the model!'''
test_loss, test_acc = model.evaluate(test_images, test_labels) # TODO
# test_loss, test_acc = # TODO

print('Test accuracy:', test_acc)

Puede observar que la precisión en el conjunto de datos de prueba es un poco menor que la precisión en el conjunto de datos de entrenamiento. Esta brecha entre la precisión del entrenamiento y la precisión de la prueba es un ejemplo de * sobreajuste *, cuando un modelo de aprendizaje automático se desempeña peor en datos nuevos que en sus datos de entrenamiento.

¿Cuál es la mayor precisión que puede lograr con este primer modelo completamente conectado? Dado que la tarea de clasificación de dígitos escritos a mano es bastante sencilla, es posible que se pregunte cómo podemos hacerlo mejor ...



## 1.3 Red neuronal convolucional (CNN) para clasificación de dígitos escritos a mano

Como vimos en la conferencia, las redes neuronales convolucionales (CNN) son particularmente adecuadas para una variedad de tareas en la visión por computadora y han logrado precisiones casi perfectas en el conjunto de datos MNIST. Ahora construiremos una CNN compuesta por dos capas convolucionales y capas agrupadas, seguidas de dos capas completamente conectadas, y finalmente generaremos una distribución de probabilidad sobre las clases de 10 dígitos (0-9). La CNN que estaremos construyendo se muestra a continuación:

![alt_text](https://raw.githubusercontent.com/aamini/introtodeeplearning/master/lab2/img/convnet_fig.png "Arquitectura CNN para clasificación MNIST")

### Definir el modelo de CNN

Usaremos los mismos conjuntos de datos de entrenamiento y prueba que antes, y procederemos de manera similar a nuestra red completamente conectada para definir y entrenar nuestro nuevo modelo de CNN. Para hacer esto, exploraremos dos capas que no habíamos encontrado antes: puede usar [`keras.layers.Conv2D`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) para defina capas convolucionales y [`keras.layers.MaxPool2D`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D) para definir las capas de agrupación. Utilice los parámetros que se muestran en la arquitectura de red anterior para definir estas capas y crear el modelo CNN.

In [ ]:
def build_cnn_model():
    cnn_model = tf.keras.Sequential([

        # TODO: Define the first convolutional layer
        tf.keras.layers.Conv2D(filters=24, kernel_size=(3,3), activation=tf.nn.relu),
        # tf.keras.layers.Conv2D('''TODO''')

        # TODO: Define the first max pooling layer
        tf.keras.layers.MaxPool2D(pool_size=(2,2)),
        # tf.keras.layers.MaxPool2D('''TODO''')

        # TODO: Define the second convolutional layer
        tf.keras.layers.Conv2D(filters=36, kernel_size=(3,3), activation=tf.nn.relu),
        # tf.keras.layers.Conv2D('''TODO''')

        # TODO: Define the second max pooling layer
        tf.keras.layers.MaxPool2D(pool_size=(2,2)),
        # tf.keras.layers.MaxPool2D('''TODO''')

        tf.keras.layers.Flatten(),

        tf.keras.layers.Dense(256, activation=tf.nn.relu),

        # TODO: Define the last Dense layer to output the classification
        # probabilities. Pay attention to the activation needed a probability
        # output
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
        # [TODO Dense layer to output classification probabilities]
    ])

    return cnn_model

cnn_model = build_cnn_model()
# Initialize the model by passing some data through
plt.imshow(np.squeeze(train_images[0]), cmap=plt.cm.binary)

print(cnn_model.predict(train_images[[0]]))
# Print the summary of the layers in the model.
print(cnn_model.summary())

In [ ]:
'''TODO: Define the compile operation with your optimizer and learning rate of choice'''
cnn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
# cnn_model.compile(optimizer='''TODO''', loss='''TODO''', metrics=['accuracy']) # TODO

Como fue el caso con el modelo completamente conectado, podemos entrenar nuestra CNN usando el método de "ajuste" a través de la API de Keras.

In [ ]:
'''TODO: Use model.fit to train the CNN model, with the same batch_size and number of epochs previously used.'''
cnn_model.fit(train_images, train_labels, batch_size=BATCH_SIZE, epochs=EPOCHS)
# cnn_model.fit('''TODO''')

Ahora que hemos entrenado el modelo, evaluémoslo en el conjunto de datos de prueba usando el método [`evaluar`](https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#evaluate):

In [ ]:
'''TODO: Use the evaluate method to test the model!'''
test_loss, test_acc = cnn_model.evaluate(test_images, test_labels)
# test_loss, test_acc = # TODO

print('Test accuracy:', test_acc)

¿Cuál es la precisión más alta que puede lograr con el modelo de CNN y cómo se compara la precisión del modelo de CNN con la precisión de la red simple completamente conectada? ¿Qué optimizadores y tasas de aprendizaje parecen ser óptimos para entrenar el modelo de CNN?

### Haz predicciones con el modelo de CNN

Con el modelo entrenado, podemos usarlo para hacer predicciones sobre algunas imágenes. La llamada a la función [`predict`](https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#predict) genera las predicciones de salida dado un conjunto de muestras de entrada.

With the model trained, we can use it to make predictions about some images. The [`predict`](https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#predict) function call generates the output predictions given a set of input samples.


In [ ]:
#print(test_images)
print(len(test_images))
predictions = cnn_model.predict(test_images)

Con esta llamada de función, el modelo ha predicho la etiqueta para cada imagen en el conjunto de prueba. Echemos un vistazo a la predicción de la primera imagen en el conjunto de datos de prueba:

In [ ]:
predictions[0]

Como puede ver, una predicción es una matriz de 10 números. Recuerde que el resultado de nuestro modelo es una distribución de probabilidad sobre las clases de 10 dígitos. Por lo tanto, estos números describen la "confianza" del modelo de que la imagen corresponde a cada uno de los 10 dígitos diferentes.

Veamos el dígito que tiene la mayor confianza para la primera imagen en el conjunto de datos de prueba:

In [ ]:
'''TODO: identify the digit with the highest confidence prediction for the first
    image in the test dataset. '''
prediction = np.argmax(predictions[0])
# prediction = # TODO

print(prediction)

Entonces, el modelo está más seguro de que esta imagen es un "???". Podemos verificar la etiqueta de prueba (recuerde, esta es la verdadera identidad del dígito) para ver si esta predicción es correcta:

In [ ]:
print("Label of this digit is:", test_labels[0])
plt.imshow(test_images[0,:,:,0], cmap=plt.cm.binary)

Visualicemos los resultados de la clasificación en el conjunto de datos MNIST. Trazaremos imágenes del conjunto de datos de prueba junto con su etiqueta predicha, así como un histograma que proporciona las probabilidades de predicción para cada uno de los dígitos:

In [ ]:
#@title Change the slider to look at the model's predictions! { run: "auto" }

image_index = 24 #@param {type:"slider", min:0, max:100, step:1}
plt.subplot(1,2,1)
mdl.lab2.plot_image_prediction(image_index, predictions, test_labels, test_images)
plt.subplot(1,2,2)
mdl.lab2.plot_value_prediction(image_index, predictions,  test_labels)

También podemos trazar varias imágenes junto con sus predicciones, donde las etiquetas de predicción correctas son azules y las etiquetas de predicción incorrectas son grises. El número proporciona el porcentaje de confianza (sobre 100) para la etiqueta predicha. ¡Tenga en cuenta que el modelo puede tener mucha confianza en una predicción incorrecta!

In [ ]:
# Plots the first X test images, their predicted label, and the true label
# Color correct predictions in blue, incorrect predictions in red
num_rows = 5
num_cols = 9
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  mdl.lab2.plot_image_prediction(i, predictions, test_labels, test_images)
  #plt.subplot(num_rows, 2*num_cols, 2*i+2)
  #mdl.lab2.plot_value_prediction(i, predictions, test_labels)


## 1.4 Entrenando el modelo 2.0

Anteriormente, usamos la llamada a la función [`fit`](https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#fit) para entrenar el modelo. Esta función es bastante intuitiva y de alto nivel, lo que es realmente útil para modelos más simples. Como puede ver, esta función abstrae muchos detalles en la llamada de entrenamiento y tenemos menos control sobre el modelo de entrenamiento, lo que podría ser útil en otros contextos.

Como alternativa a esto, podemos usar la clase [`tf.GradientTape`](https://www.tensorflow.org/api_docs/python/tf/GradientTape) para registrar las operaciones de diferenciación durante el entrenamiento, y luego llamar a la [` tf.GradientTape.gradient`](https://www.tensorflow.org/api_docs/python/tf/GradientTape#gradient) para calcular los gradientes.

Usaremos este marco para entrenar nuestro `cnn_model` usando el descenso de gradiente estocástico.

In [ ]:
# Rebuild the CNN model
cnn_model = build_cnn_model()

batch_size = 12
loss_history = mdl.util.LossHistory(smoothing_factor=0.95) # to record the evolution of the loss
plotter = mdl.util.PeriodicPlotter(sec=2, xlabel='Iterations', ylabel='Loss', scale='semilogy')
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-2) # define our optimizer

if hasattr(tqdm, '_instances'): tqdm._instances.clear() # clear if it exists

for idx in tqdm(range(0, train_images.shape[0], batch_size)):
  # First grab a batch of training data and convert the input images to tensors
  (images, labels) = (train_images[idx:idx+batch_size], train_labels[idx:idx+batch_size])
  images = tf.convert_to_tensor(images, dtype=tf.float32)

  # GradientTape to record differentiation operations
  with tf.GradientTape() as tape:
    #'''TODO: feed the images into the model and obtain the predictions'''
    logits = cnn_model(images)
    # logits = # TODO

    #'''TODO: compute the categorical cross entropy loss
    loss_value = tf.keras.backend.sparse_categorical_crossentropy(labels, logits)
    # loss_value = tf.keras.backend.sparse_categorical_crossentropy() # TODO

  loss_history.append(loss_value.numpy().mean()) # append the loss to the loss_history record
  plotter.plot(loss_history.get())

  # Backpropagation
  '''TODO: Use the tape to compute the gradient against all parameters in the CNN model.
      Use cnn_model.trainable_variables to access these parameters.'''
  grads = tape.gradient(loss_value, cnn_model.trainable_variables)
  # grads = # TODO
  optimizer.apply_gradients(zip(grads, cnn_model.trainable_variables))


## 1.5 Conclusión
En esta parte del laboratorio, tuvo la oportunidad de jugar con diferentes clasificadores MNIST con diferentes arquitecturas (solo capas completamente conectadas, CNN) y experimentar cómo los diferentes hiperparámetros afectan la precisión (tasa de aprendizaje, etc.). La siguiente parte del laboratorio explora otra aplicación de las CNN, la detección facial y algunos inconvenientes de los sistemas de inteligencia artificial en aplicaciones del mundo real, como cuestiones de sesgo.

# 2. Transfeer Learning

In [ ]:
# Importar bibliotecas necesarias
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Cargar el modelo preentrenado VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Congelar todas las capas del modelo base
for layer in base_model.layers:
    layer.trainable = False

# Añadir capas personalizadas
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)  # Asumiendo 10 clases para la clasificación de prendas

# Crear el modelo final
model = Model(inputs=base_model.input, outputs=predictions)

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo (utilice su propio conjunto de datos)
# model.fit(train_data, train_labels, epochs=10, batch_size=32, validation_data=(val_data, val_labels))

# Descongelar algunas de las últimas capas y realizar un segundo entrenamiento
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Volver a compilar el modelo y entrenar
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.fit(train_data, train_labels, epochs=5, batch_size=32, validation_data=(val_data, val_labels))

# Evaluar el modelo (utilice su propio conjunto de datos de prueba)
# test_loss, test_acc = model.evaluate(test_data, test_labels)
